In [1]:
import sys
import os

# Adiciona o diretório src ao sys.path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# Importa os módulos
from data_fetching import fetch_inmet_data

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Composição do dataset

In [3]:
df_meta_dados = pd.read_csv('../dados/meta_dados_estacoes_2023.csv')
df_meta_dados.head()

,ano,mes,data,hora,id_estacao,precipitacao_total,pressao_atm_hora,pressao_atm_max,pressao_atm_min,radiacao_global,...,temperatura_max,temperatura_min,temperatura_orvalho_max,temperatura_orvalho_min,umidade_rel_max,umidade_rel_min,umidade_rel_hora,vento_direcao,vento_rajada_max,vento_velocidade
0,2023,4,2023-04-09,06:00:00,A419,NaN,971.4,971.6,971.4,NaN,...,22.3,22.0,22.2,21.9,99.0,99.0,99.0,221.0,NaN,NaN
1,2023,4,2023-04-09,05:00:00,A313,0.0,950.6,950.8,950.4,NaN,...,23.1,22.8,22.8,22.5,99.0,98.0,99.0,149.0,3.7,2.0
2,2023,12,2023-12-21,20:00:00,A450,0.0,983.2,983.3,982.9,146.1,...,24.1,23.9,24.1,23.7,100.0,99.0,99.0,123.0,5.7,2.9
3,2023,4,2023-04-10,06:00:00,A313,0.0,951.6,951.7,951.2,NaN,...,21.7,21.3,21.6,21.1,99.0,99.0,99.0,155.0,3.2,0.0
4,2023,12,2023-12-22,01:00:00,A373,0.0,980.2,980.2,980.1,NaN,...,24.4,24.1,24.0,23.4,99.0,95.0,99.0,18.0,1.6,0.6


Vamos utilizar apenas a temperatura máxima e mínima do dia

In [4]:
# remover colunas desnecessárias
df_meta_dados_novo = df_meta_dados.drop(columns=['radiacao_global', 'temperatura_bulbo_hora', 'temperatura_orvalho_hora',
                                            'temperatura_orvalho_max', 'temperatura_orvalho_min', 'umidade_rel_max',
                                            'umidade_rel_min', 'umidade_rel_hora','mes','ano'])

df_meta_dados_novo.head()

,data,hora,id_estacao,precipitacao_total,pressao_atm_hora,pressao_atm_max,pressao_atm_min,temperatura_max,temperatura_min,vento_direcao,vento_rajada_max,vento_velocidade
0,2023-04-09,06:00:00,A419,NaN,971.4,971.6,971.4,22.3,22.0,221.0,NaN,NaN
1,2023-04-09,05:00:00,A313,0.0,950.6,950.8,950.4,23.1,22.8,149.0,3.7,2.0
2,2023-12-21,20:00:00,A450,0.0,983.2,983.3,982.9,24.1,23.9,123.0,5.7,2.9
3,2023-04-10,06:00:00,A313,0.0,951.6,951.7,951.2,21.7,21.3,155.0,3.2,0.0
4,2023-12-22,01:00:00,A373,0.0,980.2,980.2,980.1,24.4,24.1,18.0,1.6,0.6


In [5]:
# Agrupar por dia pegando a temperatura máxima e mínima
df_meta_dados_novo = df_meta_dados_novo.groupby(['id_estacao','data']).agg({'temperatura_max':'max','temperatura_min':'min'}).reset_index()
df_meta_dados_novo.head()

,id_estacao,data,temperatura_max,temperatura_min
0,A305,2023-01-01,32.4,25.9
1,A305,2023-01-02,32.6,24.4
2,A305,2023-01-03,32.6,24.6
3,A305,2023-01-04,32.7,25.2
4,A305,2023-01-05,32.9,25.5


In [6]:
# Varificando se as estações possuem todos os dias de 2023
df_meta_dados_novo['id_estacao'].value_counts().loc[lambda x : x < 364]

Series([], Name: id_estacao, dtype: int64)

Todas as estações possuem uma contagem de pelo menos 364 dias

In [7]:
df_meta_dados_novo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30576 entries, 0 to 30575
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_estacao       30576 non-null  object 
 1   data             30576 non-null  object 
 2   temperatura_max  26142 non-null  float64
 3   temperatura_min  26142 non-null  float64
dtypes: float64(2), object(2)
memory usage: 955.6+ KB


Temos uma quantidade muito grande de valores nulos, vamos criar um modelo para tentar inferir esses valores

In [8]:
df_meta_dados = pd.read_csv('../dados/meta_dados_estacoes_2023.csv')
df_meta_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 733824 entries, 0 to 733823
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ano                       733824 non-null  int64  
 1   mes                       733824 non-null  int64  
 2   data                      733824 non-null  object 
 3   hora                      733824 non-null  object 
 4   id_estacao                733824 non-null  object 
 5   precipitacao_total        510665 non-null  float64
 6   pressao_atm_hora          588148 non-null  float64
 7   pressao_atm_max           587454 non-null  float64
 8   pressao_atm_min           587451 non-null  float64
 9   radiacao_global           339095 non-null  float64
 10  temperatura_bulbo_hora    585996 non-null  float64
 11  temperatura_orvalho_hora  551192 non-null  float64
 12  temperatura_max           585332 non-null  float64
 13  temperatura_min           585328 non-null  f

In [9]:
# Converter data pro formato datetime
df_meta_dados['data'] = pd.to_datetime(df_meta_dados['data'], format='%Y-%m-%d')

# Criar a coluna com apenas o dia
df_meta_dados['dia'] = df_meta_dados['data'].dt.day

# Converter hora pra inteiro
df_meta_dados['hora'] = df_meta_dados['hora'].str.replace(':','').astype(int)
df_meta_dados['hora'] = df_meta_dados['hora']/10000
# Codificação cíclica da hora
df_meta_dados['hora_sin'] = np.sin(2 * np.pi * df_meta_dados['hora'] / 24)
df_meta_dados['hora_cos'] = np.cos(2 * np.pi * df_meta_dados['hora'] / 24)

# Inicializar o codificador de rótulos
label_encoder = LabelEncoder()

# Aplicar a codificação de rótulos
df_meta_dados['id_estacao_encoded'] = label_encoder.fit_transform(df_meta_dados['id_estacao'])

# Removendo colunas desnecessárias
df_meta_dados = df_meta_dados.drop(columns=['data','ano','id_estacao'])
df_meta_dados = df_meta_dados.dropna()
df_meta_dados.head()

,mes,hora,precipitacao_total,pressao_atm_hora,pressao_atm_max,pressao_atm_min,radiacao_global,temperatura_bulbo_hora,temperatura_orvalho_hora,temperatura_max,...,umidade_rel_max,umidade_rel_min,umidade_rel_hora,vento_direcao,vento_rajada_max,vento_velocidade,dia,hora_sin,hora_cos,id_estacao_encoded
2,12,20.0,0.0,983.2,983.3,982.9,146.1,24.0,23.9,24.1,...,100.0,99.0,99.0,123.0,5.7,2.9,21,-0.866025,5.000000e-01,75
6,12,6.0,2.6,994.7,994.9,994.6,0.0,22.4,22.1,22.5,...,99.0,99.0,99.0,154.0,3.2,1.7,22,1.000000,6.123234e-17,38
9,12,19.0,0.0,982.2,983.0,982.2,2071.8,37.3,12.3,38.1,...,24.0,22.0,22.0,111.0,7.3,3.5,23,-0.965926,2.588190e-01,22
11,12,17.0,0.0,871.6,872.7,871.6,3349.6,28.9,6.5,29.3,...,25.0,21.0,24.0,147.0,8.7,3.2,23,-0.965926,-2.588190e-01,65
12,12,15.0,0.0,969.5,970.2,969.5,3782.5,35.1,11.6,35.1,...,32.0,23.0,24.0,95.0,8.3,3.3,23,-0.707107,-7.071068e-01,32


In [10]:
# Separar variáveis independentes (X) e dependente (y)
X = df_meta_dados.drop(columns=['temperatura_max'])
y = df_meta_dados['temperatura_max']

# Dividir em conjuntos de treino e teste, mantendo os NaNs apenas no treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# AutoML - lazypredict

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from lazypredict.Supervised import LazyRegressor

# Escalonar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Inicializar o LazyRegressor
lazy_reg = LazyRegressor()

# Ajustar e avaliar os modelos
models = lazy_reg.fit(X_train_scaled, X_test_scaled, y_train, y_test)

# Visualizar resultados
print(models)

 62%|██████▏   | 26/42 [12:23<15:11, 56.95s/it]

In [ ]:
# Prever e avaliar o modelo no conjunto de teste
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R² Score: {r2}')
print(f'Root Mean Squared Error: {rmse}')